In [73]:
import pandas as pd

In [74]:
latency_one_node = pd.read_csv('../../latency/latency_data_single_node.csv')
latency_two_nodes = pd.read_csv('../../latency/latency_data_two_nodes.csv')

# something weird happens when the number of the core of the 2 nodes is the same
# in the latency_two_nodes dataframe

latency_two_nodes.head()

,Core 1,Core 2,Message Size,Latency
0,0,0,1,3967.28
1,0,0,2,3949.78
2,0,0,4,4109.33
3,0,0,8,4046.33
4,0,0,16,4060.73


In [75]:
# in the lines in which core 1 == core 2, change the value of the latency with the value of the latency for the same core 1 but core 2 = core 2 + 2 and the same message size

for index, row in latency_two_nodes.iterrows():
    if row['Core 1'] == row['Core 2'] and row['Core 2']!=22:
        # Get the new latency value from the corresponding Core 2 + 2
        new_row = latency_two_nodes[(latency_two_nodes['Core 1'] == row['Core 1']) & 
                      (latency_two_nodes['Core 2'] == (row['Core 2'] + 2)) & 
                      (latency_two_nodes['Message Size'] == row['Message Size'])]
        
        if not new_row.empty:
            latency_two_nodes.at[index, 'Latency'] = new_row['Latency'].values[0]
    elif row['Core 1'] == row['Core 2'] and row['Core 2']==22:
        # Get the new latency value from the corresponding Core 2 - 2
        new_row = latency_two_nodes[(latency_two_nodes['Core 1'] == row['Core 1']-2) & 
                      (latency_two_nodes['Core 2'] == (row['Core 2'])) & 
                      (latency_two_nodes['Message Size'] == row['Message Size'])]
        
        if not new_row.empty:
            latency_two_nodes.at[index, 'Latency'] = new_row['Latency'].values[0]

display(latency_two_nodes)

,Core 1,Core 2,Message Size,Latency
0,0,0,1,0.22
1,0,0,2,0.20
2,0,0,4,0.20
3,0,0,8,0.20
4,0,0,16,0.19
...,...,...,...,...
2360,22,22,256,0.36
2361,22,22,512,0.44
2362,22,22,1024,0.53
2363,22,22,2048,0.75


In [97]:
# define a class process: each process has an id, a core/socket and a node

class Process:
    def __init__(self, id, core, socket, node):
        self.id = id
        self.core = core
        self.socket = socket
        self.node = node

    def __str__(self):
        return f'Process {self.id} is running on core {self.core} of node {self.node}'

# function to map the processes to the cores/sockets

def map_processes(num_processes, mapping):
    processes = []
    if mapping == 'core':
        for i in range(num_processes):
            if i < 12:
                processes.append(Process(i, i, 0, 0))
            elif i < 24 and i >= 12:
                processes.append(Process(i, i, i-12, 0))
            elif i < 36 and i >= 24:
                processes.append(Process(i, i-24, 0, 1))
            elif i >= 36:
                processes.append(Process(i, i-24, 1, 1))

    elif mapping == 'socket':
        for i in range(num_processes):
            if i < 24 and i%2 == 0:
                processes.append(Process(i, None, 0, 0))
            elif i < 24 and i%2 == 1:
                processes.append(Process(i, None, 1, 0))
            elif i >=24 and i%2 == 0:
                processes.append(Process(i, None, 0, 1))
            elif i >=24 and i%2 == 1:
                processes.append(Process(i, None, 1, 1))

    elif mapping == 'node':
        for i in range(num_processes):
            if i%4 == 0:
                processes.append(Process(i, None, 0, 0))
            elif i%4 == 2:
                processes.append(Process(i, None, 1, 0))
            elif i%4 == 1:
                processes.append(Process(i, None, 0, 1))
            elif i%4 == 3:
                processes.append(Process(i, None, 1, 1))

    return processes

# function to calculate the latency of a message between two processes

def calculate_latency(process1, process2, size, latency_one_node=latency_one_node, latency_two_nodes=latency_two_nodes):
    result = pd.DataFrame()
    # Case where both processes are on the same node
    if process1.node == process2.node:
        result = latency_one_node[(latency_one_node['Core 1'] == process1.core) & 
                                  (latency_one_node['Core 2'] == process2.core) & 
                                  (latency_one_node['Message Size'] == size)]
    # Case where processes are on different nodes
    else:
        result = latency_two_nodes[(latency_two_nodes['Core 1'] == process1.core) & 
                                   (latency_two_nodes['Core 2'] == process2.core) & 
                                   (latency_two_nodes['Message Size'] == size)]
        
        # Handle special cases when result is not found
        if result.empty:
            if process2.core == 23 and process1.core == 23:
                result = latency_two_nodes[(latency_two_nodes['Core 1'] == process1.core - 1) & 
                                           (latency_two_nodes['Core 2'] == process2.core - 1) & 
                                           (latency_two_nodes['Message Size'] == size)]
            elif process2.core == 23 and process1.core % 2 == 1:
                result = latency_two_nodes[(latency_two_nodes['Core 1'] == process1.core + 1) & 
                                           (latency_two_nodes['Core 2'] == process2.core - 1) & 
                                           (latency_two_nodes['Message Size'] == size)]
            elif process1.core%2 ==1 and process2.core % 2 == 1:
                result = latency_two_nodes[(latency_two_nodes['Core 1'] == process1.core + 1) & 
                                           (latency_two_nodes['Core 2'] == process2.core + 1) & 
                                           (latency_two_nodes['Message Size'] == size)]
            

    # Extract the latency value
    if not result.empty:
        return result['Latency'].values[0]  # Return the latency value directly
    else:
        return None  # Or handle it appropriately (e.g., return 0 or an error message)


            

# function to calculate the latency of the linear algorithm

def calculate_latency_linear(size, num_processes, latency_one_node=latency_one_node, latency_two_nodes=latency_two_nodes, mapping='core'):
    total_latency = 0
    processes = map_processes(num_processes, mapping)
    for i in range(num_processes-1):
        # assuming a message is sent when the prevoius is completed, not realistic
        latency = calculate_latency(processes[0], processes[i+1], size, latency_one_node, latency_two_nodes)
        total_latency += latency
    return total_latency

def calculate_latency_binarytree(size, num_processes, latency_one_node=latency_one_node, latency_two_nodes=latency_two_nodes, mapping='core'):
    total_latency = 0
    processes = map_processes(num_processes, mapping)
    
    def send_message_tree(process_index):
        nonlocal total_latency
        # Calculate child indices
        left_child_index = 2 * process_index + 1
        right_child_index = 2 * process_index + 2
        
        # Send message to left child if it exists
        if left_child_index < num_processes:
            latency = calculate_latency(processes[process_index], processes[left_child_index], size, latency_one_node, latency_two_nodes)
            print(f'Process {process_index} sending message to process {left_child_index} with latency {latency}')
            total_latency += latency
            send_message_tree(left_child_index)  # Recursively send to left child
            
        # Send message to right child if it exists
        if right_child_index < num_processes:
            latency = calculate_latency(processes[process_index], processes[right_child_index], size, latency_one_node, latency_two_nodes)
            print(f'Process {process_index} sending message to process {right_child_index} with latency {latency}')
            total_latency += latency
            send_message_tree(right_child_index)  # Recursively send to right child
    
    # Start sending messages from the root process (process 0)
    send_message_tree(0)
    return total_latency

In [98]:
# the results are way too high
latency = calculate_latency_linear(16, 47, mapping='core')
print(latency)

processes = map_processes(47, 'core')
print(calculate_latency(processes[0], processes[46],16))

TypeError: unsupported operand type(s) for +: 'float' and 'NoneType'

In [ ]:
calculate_latency_binarytree(16, 47, mapping='core')

Process 0 sending message to process 1 with latency 0.19
Process 1 sending message to process 3 with latency 0.19
Process 3 sending message to process 7 with latency 0.19
Process 7 sending message to process 15 with latency 0.41
Process 15 sending message to process 31 with latency None


TypeError: unsupported operand type(s) for +: 'float' and 'NoneType'